<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [4]:
import pandas as pd
import psycopg2

In [5]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [6]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [100]:
# текст запроса
query_3_1 = f'''select  count (id) as cnt_vacancies
            from vacancies
         '''

In [101]:
# результат запроса
# используем метод read_sql_query, чтобы получить результаты запроса сразу в виде датафрейма
df1 = pd.read_sql_query(query_3_1, connection)
df1

C:\Users\HP\AppData\Local\Temp\ipykernel_6700\2740457354.py:3: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,cnt_vacancies
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [102]:
# текст запроса
query_3_2 = f'''select  count (id) as cnt_employers
            from EMPLOYERS
         '''

In [103]:
# результат запроса
df2 = pd.read_sql_query(query_3_2, connection)
df2

C:\Users\HP\AppData\Local\Temp\ipykernel_6700\142250957.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,cnt_employers
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [104]:
# текст запроса
query_3_3 = f'''select  count (id) as cnt_areas
            from AREAS
         '''

In [105]:
# результат запроса
df3 = pd.read_sql_query(query_3_3, connection)
df3

C:\Users\HP\AppData\Local\Temp\ipykernel_6700\1177936900.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,cnt_areas
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [106]:
# текст запроса
query_3_4 = f'''select  count (id) as cnt_industries
            from INDUSTRIES
         '''

In [107]:
# результат запроса
df4 = pd.read_sql_query(query_3_4, connection)
df4

C:\Users\HP\AppData\Local\Temp\ipykernel_6700\1823014487.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,cnt_industries
0,294


***

In [135]:
# выводы по предварительному анализу данных
merged_df1 = pd.DataFrame({'lkey': ['cnt_vacancies', 'cnt_employers', 'cnt_areas', 'cnt_industries'],
                    'count': [49197, 23501, 1362, 294]})
import plotly.express as px

fig = px.bar(
    data_frame=merged_df1,
    x=merged_df1.index,
    y='count',
    height=500, #высота
    width=700, #ширина
    title='Количество вакансий, работодателей, городов, индустрий'
)
fig.write_image('img1.png')
fig.show()
print('Вывод: Больше всего ваканский, на втором месте количество работодателей, затем идет количество городов, на последнем месте количество индустрий')

Вывод: Больше всего ваканский, на втором месте количество работодателей, затем идет количество городов, на последнем месте количество индустрий


# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [116]:
# текст запроса
query_4_1 = f'''select
                count (v.id) as cnt,
                a.name
            from areas a
            join VACANCIES v on a.id = v.area_id
            group by a.name
            order by count (v.id) desc
            limit 5
          '''

In [117]:
# результат запроса
area_vacancies = pd.read_sql_query(query_4_1, connection)
area_vacancies

C:\Users\HP\AppData\Local\Temp\ipykernel_6700\4071002479.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,cnt,name
0,5333,Москва
1,2851,Санкт-Петербург
2,2112,Минск
3,2006,Новосибирск
4,1892,Алматы


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [118]:
# текст запроса
query_4_2 = f'''select
                count (v.id) as count_vac_salary
            from VACANCIES v
            Where v.salary_from is not null or v.salary_to is not null
          '''

In [119]:
# результат запроса
salary_not_empty = pd.read_sql_query(query_4_2, connection)
salary_not_empty

C:\Users\HP\AppData\Local\Temp\ipykernel_6700\4217971214.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count_vac_salary
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [120]:
# текст запроса
query_4_3 = f'''select
                round (avg (salary_from)) as avg_low_sallary,
                round (avg (salary_to)) as avg_upper_sallary
            from VACANCIES
          '''

In [121]:
# результат запроса
avg_salary = pd.read_sql_query(query_4_3, connection)
avg_salary

C:\Users\HP\AppData\Local\Temp\ipykernel_6700\212845365.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,avg_low_sallary,avg_upper_sallary
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [122]:
# текст запроса
query_4_4 = f'''WITH sched_employment AS ( -- это основная таблица
                    SELECT
                        schedule, 
                        employment,                     
                        COUNT(id) vacancies
                    FROM
                        vacancies 
                    GROUP BY schedule, employment -- группировка по графику, а внутри по типу занятости
                )
                SELECT
                    schedule, 
                    employment,                     
                    vacancies,
                    ROUND(vacancies/(SELECT SUM(vacancies) FROM sched_employment)*100, 5) percent
                FROM
                    sched_employment
                ORDER BY vacancies DESC
             '''

In [123]:
# результат запроса
sched_employment = pd.read_sql_query(query_4_4, connection)
sched_employment

C:\Users\HP\AppData\Local\Temp\ipykernel_6700\1879418728.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,schedule,employment,vacancies,percent
0,Полный день,Полная занятость,35367,71.88853
1,Удаленная работа,Полная занятость,7802,15.85869
2,Гибкий график,Полная занятость,1593,3.23800
3,Удаленная работа,Частичная занятость,1312,2.66683
4,Сменный график,Полная занятость,940,1.91069
5,Полный день,Стажировка,569,1.15657
6,Вахтовый метод,Полная занятость,367,0.74598
7,Полный день,Частичная занятость,347,0.70533
8,Гибкий график,Частичная занятость,312,0.63419
9,Полный день,Проектная работа,141,0.28660


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [124]:
# текст запроса
query_4_5 = f'''WITH exp_how_many AS ( -- это основная таблица
                    SELECT
                        experience,
                        COUNT(id) quantity
                    FROM
                        vacancies
                    GROUP BY experience -- группировка по опыту
                )
                SELECT
                    experience,
                    quantity,
                    ROUND(quantity/(SELECT SUM(quantity) FROM exp_how_many)*100, 1) percent -- вычисляем процент от общего количества вакансий, ну и слегка округляем
                FROM
                    exp_how_many
                ORDER BY quantity -- сортируем, как заказано
            '''

In [125]:
# результат запроса
experience_vacancies = pd.read_sql_query(query_4_5, connection)
experience_vacancies

C:\Users\HP\AppData\Local\Temp\ipykernel_6700\4185389635.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,experience,quantity,percent
0,Более 6 лет,1337,2.7
1,Нет опыта,7197,14.6
2,От 3 до 6 лет,14511,29.5
3,От 1 года до 3 лет,26152,53.2


***

In [134]:
# выводы по детальному анализу вакансий

#строим график
fig = px.bar(
    data_frame=sched_employment, #датафрейм
    x="schedule", #ось x
    y="vacancies", #ось y
    color='vacancies', #расцветка в зависимости от страны
    text = 'schedule', #текст на столбцах
    orientation='v', #ориентация графика
    height=500, #высота
    width=1000, #ширина
    title='Количество вакансий по расписанию' #заголовок
)

#отображаем его
fig.show()
fig.write_image('img2.png')
print('Вывод: Большинство вакансий на полный день, наименьшее количество вакансий - вахтовый метод')

Вывод: Большинство вакансий на полный день, наименьшее количество вакансий - вахтовый метод


# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [136]:
# текст запроса
query_5_1 = f'''WITH employers AS ( -- тут создадим таблицу с количеством вакансий у каждого работодателя, с сортировкой по убыванию
                    SELECT
                        e.name,
                        COUNT(v.id) quantity
                    FROM
                        vacancies v
                        LEFT JOIN employers e ON v.employer_id = e.id -- как обычно получаем названия работодателей
                    GROUP BY e.id
                    ORDER BY quantity DESC
                )
                (SELECT -- тут возьмем 1 строку из созданной таблицы
                     name,
                     quantity,
                     1 place
                 FROM
                     employers
                 LIMIT 1)
                UNION -- тут объединим две нужные строки в новую табличку
                (SELECT -- тут возьмем 5 строку из созданной таблицы
                     name,
                     quantity,
                     5
                 FROM
                     employers
                 OFFSET 4
                 LIMIT 1)
                 ORDER BY place -- отсортируем по месту, для красоты опять же
            '''

In [137]:
# результат запроса
employer_vacancies = pd.read_sql_query(query_5_1, connection)
employer_vacancies

C:\Users\HP\AppData\Local\Temp\ipykernel_6700\101337004.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name,quantity,place
0,Яндекс,1933,1
1,Газпром нефть,331,5


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [138]:
# текст запроса
query_5_2 = f'''SELECT
                    a.name,
                    vacan.vacan_quantity,
                    employ.employ_quantity
                FROM areas a -- присоединяем слева к таблице с регионами таблицы vacan и employ по ID регионов, получим для каждого региона количество вакансий и работодателей
                     LEFT JOIN (SELECT 
                                    a.id id,
                                    COUNT(v.id) vacan_quantity
                                FROM
                                    vacancies v 
                                    LEFT JOIN areas a ON v.area_id = a.id
                                GROUP BY a.id -- таблица vacan с названиями регионов и количеством вакансий по каждому
                                ) vacan ON a.id = vacan.id
                     LEFT JOIN (SELECT
                                    a.id id,
                                    COUNT(e.id) employ_quantity
                                FROM
                                    employers e 
                                    LEFT JOIN areas a ON e.area = a.id
                                GROUP BY a.id -- таблица employ с названиями регионов и количеством работодателей в них
                                ) employ ON a.id = employ.id
                WHERE employ_quantity IS NOT NULL -- оставим только те регионы, в которых есть хотя бы один работодатель
                ORDER BY vacan_quantity DESC, employ_quantity DESC -- сортируем по количеству вакансий по убыванию, при этом все отсутствующие будут сверху, а после этой сортировки сортируем еще по количеству работодателей по убыванию и тогда в первой строке получим требуемый регион
             '''

In [139]:
# результат запроса
area_vacan_employ = pd.read_sql_query(query_5_2, connection)
area_vacan_employ

C:\Users\HP\AppData\Local\Temp\ipykernel_6700\1732355076.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name,vacan_quantity,employ_quantity
0,Россия,NaN,410
1,Казахстан,NaN,207
2,Московская область,NaN,75
3,Краснодарский край,NaN,19
4,Ростовская область,NaN,18
...,...,...,...
649,Электроугли,1.0,1
650,Белореченск,1.0,1
651,Красный Сулин,1.0,1
652,Заволжье,1.0,1


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [140]:
# текст запроса
query_5_3 = f'''SELECT
                    e.name,
                    COUNT(DISTINCT v.area_id) quantity -- количество регионов для каждого работодателя
                FROM
                    vacancies v
                    LEFT JOIN employers e ON v.employer_id = e.id -- просто добавляем слева для названия работодателя
                GROUP BY e.id -- группируем по id работодателя
                ORDER BY quantity DESC
            '''

In [141]:
# результат запроса
employ_vacan_area = pd.read_sql_query(query_5_3, connection)
employ_vacan_area

C:\Users\HP\AppData\Local\Temp\ipykernel_6700\4219517098.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name,quantity
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14901,НПП Авиатрон,1
14902,Центр дистанционных торгов,1
14903,Городские Телекоммуникационные Системы,1
14904,"Введенский, Отель",1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [142]:
# текст запроса
query_5_4 = f'''SELECT
                    COUNT(*)
                FROM
                    employers e
                    FULL JOIN employers_industries e_i ON e.id = e_i.employer_id -- полное соединение, получаем записи (строки) для каждого работодателя со всеми id сферы деятельности (построчно) или одну запись с "пустышкой" на месте id сферы деятельности (если для id работодателя нет ни одной записи в таблице employers_industries)
                WHERE 
                    e_i.industry_id IS NULL -- оставляем только записи с пустыми полями id сферы деятельности (а это как раз работодатели с отсутсвием сфер деятельности)
            '''

In [143]:
# результат запроса
employ_industies_empty = pd.read_sql_query(query_5_4, connection)
employ_industies_empty

C:\Users\HP\AppData\Local\Temp\ipykernel_6700\669153199.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [144]:
# текст запроса
query_5_5 = f'''SELECT
                    e.name
                FROM
                    employers_industries e_i
                    LEFT JOIN employers e ON e_i.employer_id = e.id -- просто добавляем названия и id работодателей
                GROUP BY e.id -- а потом группируем по id работодателя
                HAVING COUNT(e_i.industry_id) = 4 -- оставляем только тех работодателей, у которых 4 записи, т.е. 4 сферы деятельности
                ORDER BY name -- сортировка по алфавиту
                OFFSET 2 -- отступ на 2 строки
                LIMIT 1 -- вывод первой после отступа
            '''

In [145]:
# результат запроса
third_employ_4_industies = pd.read_sql_query(query_5_5, connection)
third_employ_4_industies

C:\Users\HP\AppData\Local\Temp\ipykernel_6700\3526537909.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name
0,2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [146]:
# текст запроса
query_5_6 = f'''SELECT
                    COUNT(DISTINCT employer_id) quantity -- подсчет количества уникальных, т.е. неповторяющихся id работодателей
                FROM
                    employers_industries e_i
                    LEFT JOIN industries i ON e_i.industry_id = i.id -- добавляем сами названия сфер деятельности
                WHERE i.name ~ 'Разработка программного обеспечения' -- только те работодатели, у кого указано требуемое (точно эта фраза)
             '''

In [147]:
# результат запроса
employ_certain_industry = pd.read_sql_query(query_5_6, connection)
employ_certain_industry

C:\Users\HP\AppData\Local\Temp\ipykernel_6700\703470245.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,quantity
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [1]:
# код для получения списка городов-милионников
# импорт необходимых библиотек
import requests
from bs4 import BeautifulSoup


# извлечение данных со страницы по указанному адресу
url = 'https://ru.wikipedia.org/wiki/Города-миллионеры_России'
response = requests.get(url).text

# создание объекта типа BeautifulSoup с использованием парсера lxml
# потом методом select получаем содержимое нужной таблицы с городами-миллионниками - это список
# дальше для 1 элемента списка точно также получаем список содержимого для тегов a с атрибутами href и title
soup = BeautifulSoup(response, 'lxml')
tab_content = soup.select('#mw-content-text > div.mw-parser-output > table.standard.sortable > tbody')
cities = tab_content[0].select('a[href][title]')

# в результате получено искомое, но есть два лишних элемента - две ссылки в названиях столбцов таблицы, 
# удовлетворяющих условию, нужно их удалить
# подготовим новый пустой список
city_list = []

# в цикле проверим все элементы имеющегося списка на условия - 
# не больше двух слов (вроде нет названий городов из трех и более слов) и первая буква заглавная
for i in cities:
    elem = i.text
    if (len(elem.split()) <= 2) & (elem[0].isupper()):
        city_list.append(elem)

# преобразуем список в кортеж и выведем
city_list = tuple(city_list)

print(city_list)

('Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 'Красноярск', 'Челябинск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Краснодар', 'Омск', 'Воронеж', 'Пермь', 'Волгоград')


In [7]:
# текст запроса
query_5_7 = f'''WITH tab AS (
                SELECT
                    a.name,
                    COUNT(*)::int cnt
                FROM
                    vacancies v
                    LEFT JOIN employers e ON v.employer_id = e.id
                    LEFT JOIN areas a ON v.area_id = a.id -- в таблицу вакансий добавляем названия работодателей и регионов, методом присоединения слева
                WHERE 
                    e.name LIKE 'Яндекс' -- условие: только Яндекс
                    AND a.name IN {city_list} -- условие: только из списка миллионников точнее кортежа
                GROUP BY a.id
                ) -- cоздаем таблицу tab с регионами из заданного списка (города-миллионники) и количеством вакансий для каждого, исключительно для Яндекса
                
                SELECT 
                    *
                FROM
                    tab
                UNION -- добавляем к имеющейся таблице tab общее количество вакансий по всем городам-миллионникам из нее же
                SELECT
                    'Total',
                    SUM(cnt) -- общая сумма вакансий
                FROM
                    tab
                ORDER BY cnt -- ну и сортируем по количеству вакансий по возрастанию
             '''

In [8]:
# результат запроса
yandex_million_areas = pd.read_sql_query(query_5_7, connection)
yandex_million_areas

C:\Users\HP\AppData\Local\Temp\ipykernel_22376\2363698064.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  yandex_million_areas = pd.read_sql_query(query_5_7, connection)


,name,cnt
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


***

In [10]:
# выводы по анализу работодателей
print('Вывод: Работодатели на первом и пятом месте по количеству вакансий: Яндекс и Газпромнефть. Оказалось, что 186 регионов без вакансий. Вездесущий Яндекс ожидаемо на первом месте c результатом 181 регион. Где размещены вакансии: Москва, СПб, все административные центры субъектов РФ, самые крупные города, плюс тоже самое по Казахстану и Беларуси. Стала интересна разбивка вакансий по регионам, а что там. Полная корреляция с величиной и статусом региона, первые пять: Москва 54, Санкт-Петербург 42, Екатеринбург 39, Нижний Новгород 36, Новосибирск 35. Компанию 2ГИС, она указала 4 сферы деятельности')


Вывод: Работодатели на первом и пятом месте по количеству вакансий: Яндекс и Газпромнефть. Оказалось, что 186 регионов без вакансий. Вездесущий Яндекс ожидаемо на первом месте c результатом 181 регион. Где размещены вакансии: Москва, СПб, все административные центры субъектов РФ, самые крупные города, плюс тоже самое по Казахстану и Беларуси. Стала интересна разбивка вакансий по регионам, а что там. Полная корреляция с величиной и статусом региона, первые пять: Москва 54, Санкт-Петербург 42, Екатеринбург 39, Нижний Новгород 36, Новосибирск 35. Компанию 2ГИС, она указала 4 сферы деятельности


# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [11]:
# текст запроса
query_6_1 = f'''SELECT
                    COUNT(*)
                FROM
                    vacancies v
                WHERE 
                    lower(v.name) ~* 'data' OR lower(v.name) ~* 'данн'
             '''

In [12]:
# результат запроса
data_vacancies = pd.read_sql_query(query_6_1, connection)
data_vacancies

C:\Users\HP\AppData\Local\Temp\ipykernel_22376\2935617630.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_vacancies = pd.read_sql_query(query_6_1, connection)


,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [13]:
# текст запроса
query_6_2 = f'''SELECT
                    COUNT(*)
                FROM
                    vacancies
                WHERE
                    (name ~* 'data scientist' OR 
                    name ~* 'data science' OR 
                    name ~* 'исследователь данных' OR
                    (name ~ 'ML' AND name !~* 'HTML') OR
                    name ~* 'machine learning' OR
                    name ~* 'машинн.*обучен.*') -- применение условия для соответствия вакансии дата-сайентиста
                    AND
                    (lower(name) ~* 'junior' OR
                    experience ='Нет опыта' OR
                    employment = 'Стажировка') -- применение условия для уровня 'junior': наличие элемента 'junior' в поле name (причем в любом регистре, для этого приводим название вакансии к нижнему регистру), для опыта и занятости соответствие заданным значениям
             '''

In [14]:
# результат запроса
junior_vacancies = pd.read_sql_query(query_6_2, connection)
junior_vacancies

C:\Users\HP\AppData\Local\Temp\ipykernel_22376\625073081.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  junior_vacancies = pd.read_sql_query(query_6_2, connection)


,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [15]:
# текст запроса
query_6_3 = f'''SELECT
                    COUNT(*)
                FROM
                    vacancies
                WHERE
                    (name ~* 'data scientist' OR 
                    name ~* 'data science' OR 
                    name ~* 'исследователь данных' OR
                    (name ~ 'ML' AND name !~* 'HTML') OR
                    name ~* 'machine learning' OR
                    name ~* 'машинн.*обучен.*') -- применение условия для соответствия вакансии дата-сайентиста
                    AND
                    (key_skills ~* 'SQL' OR
                    key_skills ~* 'postgres') -- соответствие условию наличия двух ключевых навыков, причем в любом месте значения поля (с учетом регистра)
             '''

In [16]:
# результат запроса
sql_vacancies = pd.read_sql_query(query_6_3, connection)
sql_vacancies

C:\Users\HP\AppData\Local\Temp\ipykernel_22376\3698489625.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sql_vacancies = pd.read_sql_query(query_6_3, connection)


,count
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [17]:
# текст запроса
query_6_4 = f'''SELECT
                    COUNT(*)
                FROM
                    vacancies
                WHERE
                    (name ~* 'data scientist' OR 
                    name ~* 'data science' OR 
                    name ~* 'исследователь данных' OR
                    (name ~ 'ML' AND name !~* 'HTML') OR
                    name ~* 'machine learning' OR
                    name ~* 'машинн.*обучен.*') -- применение условия для соответствия вакансии дата-сайентиста
                    AND
                    (key_skills ~ 'Python')-- соответствие условию наличия ключевого навыка, причем в любом месте значения поля (с учетом регистра)
             '''

In [18]:
# результат запроса
python_vacancies = pd.read_sql_query(query_6_4, connection)
python_vacancies

C:\Users\HP\AppData\Local\Temp\ipykernel_22376\100281186.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  python_vacancies = pd.read_sql_query(query_6_4, connection)


,count
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [19]:
# текст запроса
query_6_5 = f'''SELECT
                    ROUND(AVG(length(key_skills) - length(replace(key_skills, '\t', '')) + 1), 2) skills -- считаем количество символов в строке и вычитаем количество символов в строке, где удалили символы разделители '\', т.о. вычисляем количество символов разделитетелей, да + 1 (так как разделитетелей на 1 меньше чем разделяемых элементов), ну и считаем среднее с округлением до 2 знаков
                FROM
                    vacancies
                WHERE
                    name ~* 'data scientist' OR 
                    name ~* 'data science' OR 
                    name ~* 'исследователь данных' OR
                    (name ~ 'ML' AND name !~* 'HTML') OR
                    name ~* 'machine learning' OR
                    name ~* 'машинн.*обучен.*' -- уже давно известное условие соответствия вакансии дата-сайентиста
             '''

In [20]:
# результат запроса
avg_skills = pd.read_sql_query(query_6_5, connection)
avg_skills

C:\Users\HP\AppData\Local\Temp\ipykernel_22376\4266198567.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  avg_skills = pd.read_sql_query(query_6_5, connection)


,skills
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [21]:
# текст запроса
query_6_6 = f'''SELECT
                    experience,
                    ROUND(AVG((COALESCE(salary_from, salary_to) + COALESCE(salary_to, salary_from)) / 2))::int avg_salary
                FROM
                    vacancies
                WHERE
                    (name ~* 'data scientist' OR 
                    name ~* 'data science' OR 
                    name ~* 'исследователь данных' OR
                    (name ~ 'ML' AND name !~* 'HTML') OR
                    name ~* 'machine learning' OR
                    name ~* 'машинн.*обучен.*')
                    AND
                    (salary_from IS NOT NULL OR salary_to IS NOT NULL)
                GROUP BY experience
                ORDER BY avg_salary DESC
             '''

In [22]:
# результат запроса
avg_salary =pd.read_sql_query(query_6_6, connection)
avg_salary

C:\Users\HP\AppData\Local\Temp\ipykernel_22376\4147299713.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  avg_salary = pd.read_sql_query(query_6_6, connection)


,experience,avg_salary
0,От 3 до 6 лет,243115
1,От 1 года до 3 лет,139675
2,Нет опыта,74643


***

In [23]:
# выводы по предметному анализу
print('Вывод: Специальность «Data Science» достаточно редко востребована в общем количестве вакансий ИТ сферы. Вакансии предлагаются скорее крупными компаниями (банки и иные финансовые компании, телекоммуникационные корпорации, интернет-гиганты торговой, социально-сетевой, игровой и т.п. направленности, торговые гиганты и т.п.), крупные промышленные компании и научные учреждения. Востребованы специалисты с опытом, с набором определенных ключевых навыков. Вакансии предлагаются в крупных городах, Москве, СПб. Возможная удаленная работа. Заработная плата в целом в два – три раза выше, чем в среднем для ИТ сферы. И зависит в первую очередь от опыта.')

Вывод: Специальность «Data Science» достаточно редко востребована в общем количестве вакансий ИТ сферы. Вакансии предлагаются скорее крупными компаниями (банки и иные финансовые компании, телекоммуникационные корпорации, интернет-гиганты торговой, социально-сетевой, игровой и т.п. направленности, торговые гиганты и т.п.), крупные промышленные компании и научные учреждения. Востребованы специалисты с опытом, с набором определенных ключевых навыков. Вакансии предлагаются в крупных городах, Москве, СПб. Возможная удаленная работа. Заработная плата в целом в два – три раза выше, чем в среднем для ИТ сферы. И зависит в первую очередь от опыта.


# Общий вывод по проекту

In [24]:
# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования
print('Вывод: более 80 % вакансий предполагают личное присутствие на рабочем месте (полный день, сменный и гибкий график), на долю удаленной работы приходится около 19 %. Вахтовый метод примерно 0.8 %. Относительно формата занятости: более 93 % вакансий предполагают полную занятость. Частичная занятость и стажировка - всего чуть больше 4 и полутра процентов, соответственно. Проектная работа, менее 1 %. Требуемый опыт работы указан в подавляющем количестве вакансий - более 85 %. При этом половина устанавливает опыт от 1 до 3 лет, почти 30 % от 3 до 6 лет и 2.7 % - больше 6 лет опыта. Опыт прямо связан зарплатными предложениями, при росте опыта растет предлагаемая зарплата.')



Вывод: более 80 % вакансий предполагают личное присутствие на рабочем месте (полный день, сменный и гибкий график), на долю удаленной работы приходится около 19 %. Вахтовый метод примерно 0.8 %. Относительно формата занятости: более 93 % вакансий предполагают полную занятость. Частичная занятость и стажировка - всего чуть больше 4 и полутра процентов, соответственно. Проектная работа, менее 1 %. Требуемый опыт работы указан в подавляющем количестве вакансий - более 85 %. При этом половина устанавливает опыт от 1 до 3 лет, почти 30 % от 3 до 6 лет и 2.7 % - больше 6 лет опыта. Опыт прямо связан зарплатными предложениями, при росте опыта растет предлагаемая зарплата.
